In [35]:
import prettytable as prettytable
import random as rnd
POPULATION_SIZE = 20
NUMB_OF_ELITE_SCHEDULES = 5
TOURNAMENT_SELECTION_SIZE = 4
MUTATION_RATE = 0.1
class Data:
    ROOMS = [["R1",25], ["R2",45], ["R3",35]]
    MEETING_TIMES = [["MT1", "MWF 09:00 - 10:00"],
                     ["MT2", "MWF 10:00 - 11:00"],
                     ["MT3", "TTH 09:00 - 10:30"],
                     ["MT4", "TTH 10:30 - 12:00"]]
    STUDENTS = [["I1", "Calvin Anderson",20],
                   ["I2", "Caleb Bryant", 20],
                   ["I3", "Godwin Ekuma", 10],
                   ["I4", "Wenyu Zhao", 10]]
    COURSES = [["C1", "325K", 15, "Math"],
               ["C2", "319K", 8, "EE"],
               ["C3", "462k", 5, "Math"],
               ["C4", "464K", 12,"EE"], 
               ["C5", "360C", 15, "EE"],
               ["C6", "303K", 5, "PHYS"],
               ["C7", "303L", 15, "PHYS"]]


    def __init__(self):
        self._rooms = []; self._meetingTimes = []; self._students = []; self._courses = []; self._depts = []

        ##Go through for each rooms and append it to the class _rooms - Converting data to classes
        for i in range(0, len(self.ROOMS)):
            self._rooms.append(Room(self.ROOMS[i][0], self.ROOMS[i][1]))

        ##Go through for each rooms and append it to the class _meetingTimes - Converting data to classes
        for i in range(0, len(self.MEETING_TIMES)):
            self._meetingTimes.append(MeetingTime(self.MEETING_TIMES[i][0], self.MEETING_TIMES[i][1]))
        
        ##Go through for each rooms and append it to the class _students - Converting data to classes
        for i in range(0, len(self.STUDENTS)):
            self._students.append(Student(self.STUDENTS[i][0], self.STUDENTS[i][1], self.STUDENTS[i][2]))

        ##Go through for each course and append each student to the class _Course - Converting data to classes
        ##Problem here is using data instead of class instance
        for i in range(0, len(self.COURSES)):
            temp = []
            for j in range(0, len(self._students)):
                temp.append(self._students[j])
            self.COURSES[i].append(temp)


        ##Go through for each course and append it to the class _course - Converting data to classes
        for i in range(0, len(self.COURSES)):
            self._courses.append(Course(self.COURSES[i][0], self.COURSES[i][1], self.COURSES[i][2], self.COURSES[i][4]))

        dept1 = Department("MATH", [self._courses[0], self._courses[2]])
        dept2 = Department("EE", [self._courses[1], self._courses[3], self._courses[4]])
        dept3 = Department("PHY", [self._courses[5], self._courses[6]])
        self._depts = [dept1, dept2, dept3]
        self._numberOfClasses = 0
        for i in range(0, len(self._depts)):
            self._numberOfClasses += len(self._depts[i].get_courses())
    def get_rooms(self): return self._rooms
    def get_students(self): return self._students
    def get_courses(self): return self._courses
    def get_depts(self): return self._depts
    def get_meetingTimes(self): return self._meetingTimes
    def get_numberOfClasses(self): return self._numberOfClasses


In [36]:
class Schedule:
    def __init__(self):
        self._data = data
        self._classes = []
        self._numbOfConflicts = 0
        self._fitness = -1
        self._classNumb = 0
        self._isFitnessChanged = True
    def get_classes(self):
        self._isFitnessChanged = True
        return self._classes
    def get_numbOfConflicts(self): return self._numbOfConflicts
    def get_fitness(self):
        if (self._isFitnessChanged == True):
            self._fitness = self.calculate_fitness()
            self._isFitnessChanged = False
        return self._fitness
    def initialize(self):
        depts = self._data.get_depts()
        for i in range(0, len(depts)):
            courses = depts[i].get_courses()
            for j in range(0, len(courses)):
                newClass = Class(self._classNumb, depts[i], courses[j])
                self._classNumb += 1
                newClass.set_meetingTime(data.get_meetingTimes()[rnd.randrange(0, len(data.get_meetingTimes()))])
                newClass.set_room(data.get_rooms()[rnd.randrange(0, len(data.get_rooms()))])
                newClass.set_student(courses[j].get_students()[rnd.randrange(0, len(courses[j].get_students()))])
                self._classes.append(newClass)
        return self
    def calculate_fitness(self):
        self._numbOfConflicts = 0
        classes = self.get_classes()
        for i in range(0, len(classes)):
            if (classes[i].get_room().get_seatingCapacity() < classes[i].get_course().get_activityHours()):
                self._numbOfConflicts += 1
            for j in range(0, len(classes)):
                if (j >= i):
                    if (classes[i].get_meetingTime() == classes[j].get_meetingTime() and
                    classes[i].get_id() != classes[j].get_id()):
                        if (classes[i].get_room() == classes[j].get_room()): self._numbOfConflicts += 1
                        if (classes[i].get_student() == classes[j].get_student()): self._numbOfConflicts += 1
        return 1 / ((1.0*self._numbOfConflicts + 1))
    def __str__(self):
        returnValue = ""
        for i in range(0, len(self._classes)-1):
            returnValue += str(self._classes[i]) + ", "
        returnValue += str(self._classes[len(self._classes)-1])
        return returnValue


In [37]:
class Population:
    def __init__(self, size):
        self._size = size
        self._data = data
        self._schedules = []
        for i in range(0, size): self._schedules.append(Schedule().initialize())
    def get_schedules(self): return self._schedules


In [38]:
class GeneticAlgorithm:
    def evolve(self, population): return self._mutate_population(self._crossover_population(population))
    def _crossover_population(self, pop):
        crossover_pop = Population(0)
        for i in range(NUMB_OF_ELITE_SCHEDULES):
            crossover_pop.get_schedules().append(pop.get_schedules()[i])
        i = NUMB_OF_ELITE_SCHEDULES
        while i < POPULATION_SIZE:
            schedule1 = self._select_tournament_population(pop).get_schedules()[0]
            schedule2 = self._select_tournament_population(pop).get_schedules()[0]
            crossover_pop.get_schedules().append(self._crossover_schedule(schedule1, schedule2))
            i += 1
        return crossover_pop
    def _mutate_population(self, population):
        for i in range(NUMB_OF_ELITE_SCHEDULES, POPULATION_SIZE):
            self._mutate_schedule(population.get_schedules()[i])
        return population
    def _crossover_schedule(self, schedule1, schedule2):
        crossoverSchedule = Schedule().initialize()
        for i in range(0, len(crossoverSchedule.get_classes())):
            if (rnd.random() > 0.5): crossoverSchedule.get_classes()[i] = schedule1.get_classes()[i]
            else: crossoverSchedule.get_classes()[i] = schedule2.get_classes()[i]
        return crossoverSchedule
    def _mutate_schedule(self, mutateSchedule):
        schedule = Schedule().initialize()
        for i in range(0, len(mutateSchedule.get_classes())):
            if(MUTATION_RATE > rnd.random()): mutateSchedule.get_classes()[i] = schedule.get_classes()[i]
        return mutateSchedule
    def _select_tournament_population(self, pop):
        tournament_pop = Population(0)
        i = 0
        while i < TOURNAMENT_SELECTION_SIZE:
            tournament_pop.get_schedules().append(pop.get_schedules()[rnd.randrange(0, POPULATION_SIZE)])
            i += 1
        tournament_pop.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
        return tournament_pop


In [39]:
class Course:
    def __init__(self, number, name, activityHours, students,):
        self._number = number
        self._name = name
        self._activityHours = activityHours
        self._students = students
    def get_number(self): return self._number
    def get_name(self): return self._name
    def get_students(self): return self._students
    def get_activityHours(self): return self._activityHours
    def __str__(self): return self._name
class Student:
    def __init__(self, id, name, hrsAvailable):
        self._id = id
        self._name = name
        self._hrsAvailable = hrsAvailable
    def get_id(self): return self._id
    def get_name(self): return self._name
    def get_hrsAvailable(self): return self._hrsAvailable
    def __str__(self): return self._name
class Room:
    def __init__(self, number, seatingCapacity):
        self._number = number
        self._seatingCapacity = seatingCapacity
    def get_number(self): return self._number
    def get_seatingCapacity(self): return self._seatingCapacity
class MeetingTime:
    def __init__(self, id, time):
        self._id = id
        self._time = time
    def get_id(self): return self._id
    def get_time(self): return self._time
class Department:
    def __init__(self, name, courses):
        self._name = name
        self._courses = courses
    def get_name(self): return self._name
    def get_courses(self): return self._courses


In [40]:
class Class:
    def __init__(self, id, dept, course):
        self._id = id
        self._dept = dept
        self._course = course
        self._student = None
        self._meetingTime = None
        self._room = None
    def get_id(self): return self._id
    def get_dept(self): return self._dept
    def get_course(self): return self._course
    def get_student(self): return self._student
    def get_meetingTime(self): return self._meetingTime
    def get_room(self): return self._room
    def set_student(self, student): self._student = student
    def set_meetingTime(self, meetingTime): self._meetingTime = meetingTime
    def set_room(self, room): self._room = room
    def __str__(self):
        return str(self._dept.get_name()) + "," + str(self._course.get_number()) + "," + \
               str(self._room.get_number()) + "," + str(self._student.get_id()) + "," + str(self._meetingTime.get_id())


In [41]:
class DisplayMgr:
    def print_available_data(self):
        print("> All Available Data")
        self.print_dept()
        self.print_course()
        self.print_room()
        self.print_student()
        self.print_meeting_times()
    def print_dept(self):
        depts = data.get_depts()
        availableDeptsTable = prettytable.PrettyTable(['dept', 'courses'])
        for i in range(0, len(depts)):
            courses = depts.__getitem__(i).get_courses()
            tempStr = "["
            for j in range(0, len(courses) - 1):
                tempStr += courses[j].__str__() + ", "
            tempStr += courses[len(courses) - 1].__str__() + "]"
            availableDeptsTable.add_row([depts.__getitem__(i).get_name(), tempStr])
        print(availableDeptsTable)
    def print_course(self):
        availableCoursesTable = prettytable.PrettyTable(['id', 'course #', 'Activity Hours', 'students'])
        courses = data.get_courses()
        for i in range(0, len(courses)):
            students = courses[i].get_students()
            tempStr = ""
            for j in range(0, len(students) - 1):
                tempStr += students[j].__str__() + ", "
            tempStr += students[len(students) - 1].__str__()
            availableCoursesTable.add_row(
                [courses[i].get_number(), courses[i].get_name(), str(courses[i].get_activityHours()), tempStr])
        print(availableCoursesTable)
    def print_student(self):
        availableStudentsTable = prettytable.PrettyTable(['id', 'student'])
        students = data.get_students()
        for i in range(0, len(students)):
            availableStudentsTable.add_row([students[i].get_id(), students[i].get_name()])
        print(availableStudentsTable)
    def print_room(self):
        availableRoomsTable = prettytable.PrettyTable(['room #', 'max seating capacity'])
        rooms = data.get_rooms()
        for i in range(0, len(rooms)):
            availableRoomsTable.add_row([str(rooms[i].get_number()), str(rooms[i].get_seatingCapacity())])
        print(availableRoomsTable)
    def print_meeting_times(self):
        availableMeetingTimeTable = prettytable.PrettyTable(['id', 'Meeting Time'])
        meetingTimes = data.get_meetingTimes()
        for i in range(0, len(meetingTimes)):
            availableMeetingTimeTable.add_row([meetingTimes[i].get_id(), meetingTimes[i].get_time()])
        print(availableMeetingTimeTable)
    def print_generation(self, population):
        table1 = prettytable.PrettyTable(['schedule #', 'fitness', '# of conflicts', 'classes [dept,class,room,student,meeting-time]'])
        schedules = population.get_schedules()
        for i in range(0, len(schedules)):
            table1.add_row([str(i), round(schedules[i].get_fitness(),3), schedules[i].get_numbOfConflicts(), schedules[i].__str__()])
        print(table1)
    def print_schedule_as_table(self, schedule):
        classes = schedule.get_classes()
        table = prettytable.PrettyTable(['Class #', 'Dept', 'Course (number, Activity Hours)', 'Room (Capacity)', 'Student (Id)',  'Meeting Time (Id)'])
        for i in range(0, len(classes)):
            table.add_row([str(i), classes[i].get_dept().get_name(), classes[i].get_course().get_name() + " (" +
                           classes[i].get_course().get_number() + ", " +
                           str(classes[i].get_course().get_activityHours()) +")",
                           classes[i].get_room().get_number() + " (" + str(classes[i].get_room().get_seatingCapacity()) + ")",
                           classes[i].get_student().get_name() +" (" + str(classes[i].get_student().get_id()) +")",
                           classes[i].get_meetingTime().get_time() +" (" + str(classes[i].get_meetingTime().get_id()) +")"])
        print(table)


In [42]:
data = Data()
displayMgr = DisplayMgr()
displayMgr.print_available_data()
generationNumber = 0
print("\n> Generation # "+str(generationNumber))
population = Population(POPULATION_SIZE)
population.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
displayMgr.print_generation(population)
displayMgr.print_schedule_as_table(population.get_schedules()[0])
geneticAlgorithm = GeneticAlgorithm()
while (population.get_schedules()[5].get_fitness() != 1.0):
    generationNumber += 1
    print("\n> Generation # " + str(generationNumber))
    population = geneticAlgorithm.evolve(population)
    population.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
    displayMgr.print_generation(population)
    displayMgr.print_schedule_as_table(population.get_schedules()[0])
print("\n\n")



> All Available Data
+------+--------------------+
| dept |      courses       |
+------+--------------------+
| MATH |    [325K, 462k]    |
|  EE  | [319K, 464K, 360C] |
| PHY  |    [303K, 303L]    |
+------+--------------------+
+----+----------+----------------+---------------------------------------------------------+
| id | course # | Activity Hours |                         students                        |
+----+----------+----------------+---------------------------------------------------------+
| C1 |   325K   |       15       | Calvin Anderson, Caleb Bryant, Godwin Ekuma, Wenyu Zhao |
| C2 |   319K   |       8        | Calvin Anderson, Caleb Bryant, Godwin Ekuma, Wenyu Zhao |
| C3 |   462k   |       5        | Calvin Anderson, Caleb Bryant, Godwin Ekuma, Wenyu Zhao |
| C4 |   464K   |       12       | Calvin Anderson, Caleb Bryant, Godwin Ekuma, Wenyu Zhao |
| C5 |   360C   |       15       | Calvin Anderson, Caleb Bryant, Godwin Ekuma, Wenyu Zhao |
| C6 |   303K   |       5